# LA's COVID-19 Reopening / Recovery Indicators 

The NY Times wrote about rushed reopenings being [a global trial-and-error experiment](https://www.nytimes.com/2020/05/07/world/europe/coronavirus-reopening-costs.html) and a further analysis showing how US states reopening [fail to meet the WH guidelines](https://www.nytimes.com/interactive/2020/05/07/us/coronavirus-states-reopen-criteria.html). Vox has written about [the importance of COVID testing indicators](https://www.vox.com/2020/5/8/21249880/coronavirus-testing-covid-reopening-economy-lockdowns-social-distancing) as a prerequisite for reopening. 

While we wait for LA's own reopening indicators, we can still see how LA stacks up against NY and Chicago's indicators. [NY State's Reopening Indicators](https://www.nytimes.com/2020/05/04/nyregion/coronavirus-reopen-cuomo-ny.html) were based heavily in part by the White House [Opening Up America Again gating criteria](https://www.whitehouse.gov/wp-content/uploads/2020/04/Guidelines-for-Opening-Up-America-Again.pdf), and Chicago outlined even [tougher benchmarks than the IL statewide strategy](https://www.chicagotribune.com/coronavirus/ct-coronavirus-chicago-reopening-lightfoot-20200508-ztpnouwexrcvfdfcr2yccbc53a-story.html). In terms of outbreak severity, [LA and Chicago come right after NYC](https://www.nytimes.com/2020/05/09/us/coronavirus-chicago.html).

## [White House Gating Criteria](https://www.whitehouse.gov/wp-content/uploads/2020/04/Guidelines-for-Opening-Up-America-Again.pdf)
### Symptoms
* Downward trajectory of influenza-like illnesses (ILI) reported within a 14-day period **and**
* Downward trajectory of COVID-like syndromic cases reported within a 14-day period

### Cases
* Downward trajectory of documented cases within a 14-day period **or**
* Downward trajectory of positive tests as a percent of total tests within a 14-day period (flat or increasing volume of tests) 

### Hospitals
* Treat all patients without crisis care **and**
* Robust testing program in place for at-risk healthcare workers, including emerging antibody testing

| NY | Chicago |
| --- | --- |
| At least 14 days decline in total net hospitalizations on a 3-day rolling average **or** <BR> Less than 15 net new total hospitalizations on a 3-day rolling average </BR> | At least 14 days decline in new cases, on average 
| At least 14 days decline in total net deaths on a 3-day rolling average **or** <BR> Less than 5 new deaths on a 3-day rolling average </BR> | At least 14 days decline in of case rate (per capita?) |
| At least 30% total hospital and ICU beds available  | At least 14 days decline in rate of new cases (per capita?)
| At least 90 days of PPE stockpiled  | Stable or declining rates of hospitalizations  
| Capacity to conduct 30 tests per 1,000 residents per month  | Stable or declining rates of ICU admissions 
| 30 working contact tracers per 100,000 residents | Stable or declining rates of ICU deaths
| Rural regions: Fewer than 2 new COVID patients admitted per 100,000 residents per day | Share of positive results are decreasing below 15% of those tested in the community 
|   | Fewer than 1,800 coronavirus patients in hospital beds 
|   | Fewer than 600 coronavirus patients in ICU beds
|   | Fewer than 450 coronavirus patients on ventilators
|   | Test 135,000 per month, equivalent to 4,500 people per day

In [1]:
import altair as alt
import numpy as np
import pandas as pd
import pytz
import utils

from datetime import date, datetime, timedelta
from IPython.display import Markdown

US_COUNTY_URL = "http://lahub.maps.arcgis.com/sharing/rest/content/items/782ca660304a4bdda1cc9757a2504647/data"
LA_CITY_URL = "https://lahub.maps.arcgis.com/sharing/rest/content/items/7175fba373f541a7a19df56b6a0617f4/data"
TESTING_URL = "http://lahub.maps.arcgis.com/sharing/rest/content/items/158dab4a07b04ecb8d47fea1746303ac/data"
HOSPITAL_URL = "http://lahub.maps.arcgis.com/sharing/rest/content/items/3da1eb3e13a14743973c96b945bd1117/data"
CROSSWALK_URL = (
    "https://raw.githubusercontent.com/CityOfLosAngeles/aqueduct/master/dags/public-health/covid19/"
    "msa_county_pop_crosswalk.csv"
)

# Default parameters
county_state_name = "Los Angeles, CA"
state_name = "California"
msa_name = "Los Angeles-Long Beach-Anaheim, CA"
time_zone = "US/Pacific"
start_date = "4/1/20"
yesterday_date = (
    (datetime.today()
                .astimezone(pytz.timezone(f'{time_zone}'))
                .date()
        - timedelta(days=1)
    ).strftime("%-m/%-d/%y")
)
today_date = (
    datetime.today()
             .astimezone(pytz.timezone(f'{time_zone}'))
             .date()
             .strftime("%-m/%-d/%y")
)

In [ ]:
la_county = utils.case_indicators_county("Los Angeles, CA", "3/1/20")
la_city = utils.case_indicators_lacity("5/1/20")
#ca = utils.case_indicators_state("California", "3/1/20")
#la_msa = utils.case_indicators_msa("Los Angeles", "3/1/20")

In [2]:
tests_df = pd.read_csv(TESTING_URL)
cases_df = pd.read_csv(LA_CITY_URL)

#  Merge and rename columns
df = pd.merge(cases_df, tests_df, on = "Date", how = "left")

keep_cols = [
    "Date",
    "City of LA Cases",
    "City of LA New Cases",
    "Performed", 
    "Cumulative",
]

df = (df[keep_cols].assign(
        Date = pd.to_datetime(df.Date),
    )
      .rename(columns = 
              {"City of LA Cases": "cases",
               "City of LA New Cases": "new_cases",
               "Performed": "new_tests",
               "Cumulative": "tests",
               "Date": "date"}) 
)

# Subset to particular start and end date
df = (df[(df.date >= start_date) & (df.date <= yesterday_date)]
      .sort_values("date")
     )

df = df.assign(
        cases = df.cases.astype(int),
        new_cases = df.new_cases.astype(int),
        new_tests = df.new_tests.astype(int),
        tests = df.tests.astype(int),
        week = pd.to_datetime(df.date).dt.week,
) 

# Aggregate to the week
weekly_total = (df.groupby("week")
            .agg({"new_cases":"sum", 
                  "new_tests":"sum",
                  "date": "min",
                  "cases":"count",
                 })
            .reset_index()
            .rename(columns = {"new_cases":"weekly_cases", 
                               "new_tests":"weekly_tests", 
                               "date": "start_of_week", 
                               "cases":"days_counted"})
           )

df = pd.merge(df, weekly_total, on = "week", how = "inner")

keep_col = [
    "week", 
    "start_of_week",
    "weekly_cases",
    "weekly_tests",
]

# Calculate share of positive results
df = (df[df.days_counted==7][keep_col].drop_duplicates()
    .assign(
        pct_positive = df.weekly_cases / df.weekly_tests,
    )
)

In [3]:
bar = (
        alt.Chart(df)
        .mark_bar(color="navy", )
        .encode(
            x=alt.X(
                "yearmonthdate(start_of_week)",
                title="date",
                axis=alt.Axis(format="%-m/%-d/%y"),
            ),
            y=alt.Y(
                "pct_positive", 
                title="Share of Positive COVID-19 Results",
            ),
        )
    )

line = (
    alt.Chart(df)
    .mark_line(color="red", strokeDash=[5, 2])
    .encode(
        x=alt.X("yearmonthdate(start_of_week)"),
        y=alt.Y("weekly_tests", title="# Weekly Tests")
    )
)



positive_tests = (
    alt.concat(
        alt.layer(bar, line, title="Weekly Test Volume and Share of Positive Results")
            .resolve_scale(y='independent',x='shared')
        )
)

positive_tests

alt.ConcatChart(...)

## Testing Indicators

In [ ]:
daily_tests = utils.testing_lacity('4/1/20', 'daily', 10_000, 16_667)
monthly_tests = utils.testing_lacity('4/1/20', 'monthly', 300_000, 500_000)

In [ ]:


lower_bound = 10_000
upper_bound = 16_667

def testing_meet_indicators(yesterday_date, lower_bound, upper_bound):
    """
    Returns a red or green indicator based on yesterday's results.
    """
    # New cases (which day out of 14 has it been declining?)
    # New deaths (which day out of 14 has it been declining?)
    # Hospital capacity

    # Daily testing
    test_indicator = daily_tests[daily_tests.Date==yesterday_date].iloc[0]["Performed"]

    if test_indicator < lower_bound:
        return "red"
    elif test_indicator >= lower_bound:
        return "green"
    elif test_indicator >= upper_bound:
        return "blue"
    
test_indicator_color = testing_meet_indicators(yesterday_date, lower_bound, upper_bound)
#print(test_indicator_color)
display(Markdown(f"<span style='color:{test_indicator_color}'> Daily Testing </span>"))

## Hospital Capacity Indicators

In [ ]:
hospital = utils.hospital_capacity_lacity("4/1/20")

# Markdown Red/ Green Indicator 
* The next cell displays in the color red or green 

In [ ]:
def red_green():
    """
    returns a random "red" or "green" text in markdown.
    """
    return np.random.choice(['green', 'red'])

color = red_green()
print(color)
display(Markdown(f'<span style="color:{color}">some *blue* text</span>.'))

In [ ]:
"""To convert to HTML: 
jupyter nbconvert --to html --no-input --no-prompt my-notebook.ipynb
"""